<a href="https://colab.research.google.com/github/shraddha-an/nn-practice/blob/main/tf_regression_nn_car.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **TensorFlow Regression Neural Network**

In [ ]:
# Importing libraries
# Data Manipulation/Handling
import pandas as pd
import numpy as np

# Time
from datetime import datetime as dt

# TensorFlow
import tensorflow as tf

# Plotting 
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
# Importing dataset
ds = pd.read_csv('car_data.csv')
ds.head()

,Car_Name,Year,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner
0,ritz,2014,3.35,5.59,27000,Petrol,Dealer,Manual,0
1,sx4,2013,4.75,9.54,43000,Diesel,Dealer,Manual,0
2,ciaz,2017,7.25,9.85,6900,Petrol,Dealer,Manual,0
3,wagon r,2011,2.85,4.15,5200,Petrol,Dealer,Manual,0
4,swift,2014,4.60,6.87,42450,Diesel,Dealer,Manual,0


In [ ]:
# Feature Engineering: Calculating Age based on the Year column
ds['Age'] = ds['Year'].apply(lambda x: dt.today().year - x)

# Dropping car_name & year columns
ds.drop(columns = ['Car_Name', 'Year'], inplace = True)

ds.head()

,Selling_Price,Present_Price,Kms_Driven,Fuel_Type,Seller_Type,Transmission,Owner,Age
0,3.35,5.59,27000,Petrol,Dealer,Manual,0,7
1,4.75,9.54,43000,Diesel,Dealer,Manual,0,8
2,7.25,9.85,6900,Petrol,Dealer,Manual,0,4
3,2.85,4.15,5200,Petrol,Dealer,Manual,0,10
4,4.60,6.87,42450,Diesel,Dealer,Manual,0,7


In [ ]:
# Splitting into X/y matrices; features and target matrices
X = ds.iloc[:, 1:].values
y = ds.iloc[:, 0].values.reshape(-1, 1)

# Splitting into train/test subsets
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.2)

X_train[34]

array([0.57, 25000, 'Petrol', 'Individual', 'Manual', 1, 9], dtype=object)

In [ ]:
# Label Encoding: Fuel_Type, Seller, Transmission columns
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer

enc = LabelEncoder()

X_train[:, 2] = enc.fit_transform(X_train[:, 2])
X_test[:, 2] = enc.transform(X_test[:, 2])

X_train[:, 3] = enc.fit_transform(X_train[:, 3])
X_test[:, 3] = enc.transform(X_test[:, 3])

X_train[:, 4] = enc.fit_transform(X_train[:, 4])
X_test[:, 4] = enc.transform(X_test[:, 4])

# One Hot Encoding the Fuel_Type column
ct = ColumnTransformer(transformers = [('one_hot_encoder', OneHotEncoder(), [2])],
                       remainder = 'passthrough')

X_train = ct.fit_transform(X_train)
X_test = ct.transform(X_test)

# Applying Standardization
from sklearn.preprocessing import StandardScaler

sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [ ]:
# Converting to a TF dataset
from tensorflow.data import Dataset

train = Dataset.from_tensor_slices((X_train, y_train))
train = train.batch(batch_size = 16)

test = Dataset.from_tensor_slices((X_test, y_test))
test = test.batch(batch_size = len(X_test))

In [ ]:
# Defining the Regression Neural Network
from tensorflow.keras import Model, losses, metrics, optimizers
from tensorflow.keras.layers import Dense

# Defining a class to implement my Neural Network architecture
class RegressionNN(Model):
  # Defining layer architecture in the constructor
  def __init__(self):
    # Calling all the parent constructors
    super().__init__()

    # Layers
    self.a = Dense(units = 100, activation = 'relu')
    self.b = Dense(units = 200, activation = 'relu')
    self.c = Dense(units = 1)

  # Defining the computations to be performed at each forward pass
  def call(self, x):
    x = self.a(x)
    x = self.b(x)
    x = self.c(x)

    return x

# Creating an object of the RegressionNN class
model = RegressionNN()

# Setting up optimizer & loss object
optimizer = optimizers.Adam()
loss_object = losses.MeanAbsoluteError()

In [ ]:
# Setting up loops to train
epochs = 20

for epoch in range(epochs):
  print('Epoch {} of {}'.format(epoch + 1, epochs))

  # Calculating train loss for every epoch
  train_loss = 0.0

  # Inner loop to iteratively feed batches of training data to the model
  for x, y in train:

    # Using tf.GradientTape() to track all computations performed on the parameters & enable auto_differentiation
    with tf.GradientTape() as tape:
      
      # Forward pass
      output = model(x)

      # Compute loss
      loss = loss_object(y, output)
      train_loss += loss.numpy()

    # Retrieving gradients of the trainable parameters wrt loss
    grads = tape.gradient(loss, model.trainable_weights)

    # Update weights to minimize loss and run 1 step of gradient descent
    optimizer.apply_gradients(zip(grads, model.trainable_weights))

  # Printing train_loss for every epoch
  print('Average train loss: {}\n'.format(train_loss/len(train)))

Epoch 1 of 20
Average train loss: 3.892195741335551

Epoch 2 of 20
Average train loss: 2.8296841859817503

Epoch 3 of 20
Average train loss: 1.7608484347661337

Epoch 4 of 20
Average train loss: 1.3433562914530437

Epoch 5 of 20
Average train loss: 1.0150699059168498

Epoch 6 of 20
Average train loss: 0.869933287302653

Epoch 7 of 20
Average train loss: 0.787440158923467

Epoch 8 of 20
Average train loss: 0.7414233068625132

Epoch 9 of 20
Average train loss: 0.7121342897415162

Epoch 10 of 20
Average train loss: 0.6848092953364054

Epoch 11 of 20
Average train loss: 0.6626728693644206

Epoch 12 of 20
Average train loss: 0.6340826213359833

Epoch 13 of 20
Average train loss: 0.6144975841045379

Epoch 14 of 20
Average train loss: 0.5890875399112702

Epoch 15 of 20
Average train loss: 0.594816917181015

Epoch 16 of 20
Average train loss: 0.5778481821219127

Epoch 17 of 20
Average train loss: 0.5450554013252258

Epoch 18 of 20
Average train loss: 0.5335821410020193

Epoch 19 of 20
Average 

In [ ]:
# Test loop

# Metrics to calculate
test_mae = metrics.MeanAbsoluteError()
test_mse = metrics.MeanSquaredError()
test_msle = metrics.MeanSquaredLogarithmicError()
test_mape = metrics.MeanAbsolutePercentageError()

for x, y in test:

  # Get outputs
  outputs = model(x)

  # Calculate & update metrics
  test_mae.update_state(y, outputs)
  test_mse.update_state(y, outputs)
  test_msle.update_state(y, outputs)
  test_mape.update_state(y, outputs)

  # Gather the results
  mae = test_mae.result().numpy()
  mse = test_mse.result().numpy()
  msle = test_msle.result().numpy()
  mape = test_mape.result().numpy()


# Update metrics dict
metrics = {'MAE': mae, 'MSE': mse, 'MSLE': msle, 'MAPE': mape}
metrics

{'MAE': 0.6963639, 'MAPE': 18.441994, 'MSE': 1.1520668, 'MSLE': 0.018023143}